In [1]:
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.embeddings import OpenAIEmbeddings

## Load the documents

In [2]:
loader = DirectoryLoader('../test_files', loader_cls=TextLoader)
docs = loader.load()

In [3]:
docs[0].page_content

'import { FC, useCallback } from "react";\nimport { Response, Error } from "./api/items";\nimport Breadcrumb from "../components/breadcrumb";\nimport ProductsList from "../components/productsList";\nimport { server } from "../../config";\nimport { useRouter } from "next/router";\nimport Head from "next/head";\n\ninterface Props {\n  data: Response | Error;\n  search: string;\n}\n\nconst Items: FC<Props> = ({ data, search }) => {\n  const router = useRouter();\n\n  if ("error" in data) {\n    return <h1>There was an error</h1>;\n  }\n\n  const handleClick = useCallback(\n    (categoryName) => {\n      const selectedCategory = data.categories_filter.find(\n        (category) => category.name === categoryName\n      );\n      if (selectedCategory) {\n        router.push(`/items?search=${search}&category=${selectedCategory.id}`);\n      }\n    },\n    [data, router]\n  );\n\n  return (\n    <>\n      {search && (\n        <Head>\n          <title>{search} | Mercado Libre</title>\n         

## Generate a document description

In [17]:
import os
import openai

# Load your API key from an environment variable or secret management service
openai.api_key = os.getenv("OPENAI_API_KEY")

chat_completion = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=[{"role": "user", "content": f"What is the purpose of the following file in a project? {docs[0].page_content}"}])

In [51]:
doc_description = chat_completion.choices[0].message.content
print(doc_description)

The purpose of this file is to render a page that displays a list of products based on a search query and category. 

- The file imports necessary components and libraries from React and Next.js.
- It defines the Props interface for the component. The Props interface includes the data and search properties, which are used to display the search results and the search query.
- The component, Items, is defined as a functional component that takes in the Props interface as its argument. Inside the component, the useRouter hook is used to access the current router object.
- If the "error" property exists in the data object, it returns an error message.
- The handleClick function is defined using the useCallback hook. It takes in a categoryName argument and finds the corresponding category in the data object. If the category is found, it updates the search query and category in the URL using router.push.
- The component returns a title and meta description for SEO purposes if there is a sear

## Create an embedding with the description

In [30]:
embeddings_model = OpenAIEmbeddings(disallowed_special=set([]))
embeddings = embeddings_model.embed_documents(
    [doc_description],
)
len(embeddings), len(embeddings[0])

(1, 1536)

In [32]:
description_embedding = embeddings[0]

## Create the vector store

In [39]:
from langchain.vectorstores import FAISS


db = FAISS.from_texts([doc_description], embeddings_model)

## Create the steps to solve a task

In [33]:
task = "Add a pagination feature to the items list, listing 2 items per page."

In [41]:
retriever = db.as_retriever()

In [42]:
relevant_docs = retriever.get_relevant_documents(task)

In [46]:
most_relevant = relevant_docs[0]

In [47]:
chat_plan = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=[{"role": "user", "content": f"Give me a detailed description of how I can accomplish the following task: {task}. This is the description of the file where I will be working: {most_relevant}"}])

In [50]:
print(chat_plan.choices[0].message.content)

To add a pagination feature to the items list with 2 items per page, you can follow these steps:

1. Declare the necessary state variables in the functional component:

```jsx
const [currentPage, setCurrentPage] = useState(1);
const itemsPerPage = 2;
```

2. Modify the rendering logic of the Items component to display the items based on the current page:

```jsx
const indexOfLastItem = currentPage * itemsPerPage;
const indexOfFirstItem = indexOfLastItem - itemsPerPage;
const currentItems = data.slice(indexOfFirstItem, indexOfLastItem); // Assuming data is an array of items

return (
  // Render the items in the currentItems array
);
```

3. Render pagination buttons to allow the user to navigate between pages:

```jsx
const totalPages = Math.ceil(data.length / itemsPerPage);

const handlePageChange = (pageNumber) => {
  setCurrentPage(pageNumber);
};

return (
  // Render the items
  // Render pagination buttons
);
```

4. Define a helper function to generate the pagination buttons:

`